In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import interp1d
from scipy import interpolate

read the data from the file

In [5]:
data = []
#/content/drive/MyDrive/21cm project/new_21cmfast_results/21cmfast210822_noLyalpha_batchNum_1_.pk
for i in range(1,12):
  path = '/content/drive/Shareddrives/Hovav/psResults/21cmfast260922_all_params_batchNum_' + str(i)
  path +=  '_.pk' if i<10 else '.pk' 
  f = open(path , 'rb')
  try:
    while True:
      tmp = pickle.load(f)
      for record in tmp.items():
        data+=[record]      
  except Exception:   
    f.close() 

In [ ]:
data = []

path = '/content/drive/MyDrive/21cm project/21cmfast results/21cmfast040722vsLyaHeating_change_only_E_0.pk'
f = open(path , 'rb')
try:
  while True:
    tmp = pickle.load(f)
    for record in tmp.items():
      data+=[record]      
except Exception:   
  f.close() 

In [ ]:
data = []
for i in range(1, 17):
  path = '/content/drive/Shareddrives/Hovav/psResults/21cmfast130822_change_only_lxvar_ID=' + str(i) + '.pk'
  f = open(path , 'rb')
  tmp = pickle.load(f)
  for record in tmp.items():
      data+=[record]      

In [ ]:
print(data[0][1].keys())
print(data[0][1]['z_PS'])
#print(data[0][1]['astro_params'])
tau = data[0][1]['tau']
#print(data[0][1]['global_xH'])
print(tau.value)
# for cc in data:
#   print(cc[1]['global_xHI'])
counter = 0
#print(data[0][1]['data_PS'])
for item in data:
  s = item[1]['data_PS']
  flag = False
  for val in s[4:]:
    if np.array_equal([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],val['delta']):
      counter+=1
      flag = True
  if flag:
    print(item[1]['astro_params'])
    print(item[1]['global_xH'])
    break
print('counter: ',counter)

dict_keys(['astro_params', 'global_xH', 'tau', 'label', 'z_global', 'data_PS', 'z_PS'])
[ 5.17970856  5.56821179  5.99898306  6.46809759  6.98576608  7.56532372
  8.20305867  8.91459909  9.71183486 10.60914901 11.63545217 12.79112694
 14.11298769 15.65179459 17.41838151 19.4822116  21.91399067 24.80730313
 28.32600921 32.57359394]
0.03007931699458617
{'ALPHA_STAR': 0.5, 'ALPHA_ESC': -0.3, 't_STAR': 0.6, 'F_STAR10': -0.8438797831498679, 'F_ESC10': -0.0741504990089612, 'L_X': 39.47385957837735, 'M_TURN': 9.823317115225514, 'NU_X_THRESH': 1183.2032879749672}
[0.99979782 0.999798   0.99979854 0.99979872 0.99980068 0.99980086
 0.9998014  0.99980158 0.9998017  0.99980187 0.99980241 0.99980259
 0.99980456 0.99980474 0.99980527 0.99980539 0.99981308 0.99981326
 0.99981344 0.99981391 0.99981409 0.99981594 0.99981612 0.99981666
 0.99981678 0.99981689 0.99981695 0.99981713 0.99981737 0.99980986
 0.99980569 0.99979782 0.99978667 0.99976802 0.99973893 0.99970198
 0.99963665 0.99954563 0.99943793 0.

In [6]:
def extractPS(data_PS,
                             z_PS,
                             z_val):
  # Find the nearest points in z_PS to z_val
    z_ind_p = np.where(z_PS >= z_val)[0][0]
    z_ind_m = z_ind_p-1
    z_p = z_PS[z_ind_p]
    z_m = z_PS[z_ind_m]
    # Extract the power spectra at the nearest points
    k_data_p = data_PS[z_ind_p]['k']
    k_data_m = data_PS[z_ind_m]['k']
    Delta_data_p = data_PS[z_ind_p]['delta']
    Delta_data_m = data_PS[z_ind_m]['delta']
    # Interpolate Delta_data_m at the values of k_data_p
    not_nan_inds = ~np.isnan(Delta_data_m)
    Delta_21_sq_interp = interp1d(k_data_m[not_nan_inds],
                                  Delta_data_m[not_nan_inds],
                                  kind='cubic',bounds_error=False)
    Delta_data_m = Delta_21_sq_interp(k_data_p)
    # Interpolate linearly between z_m and z_p
    Delta_data = (Delta_data_p*(z_val-z_m) + Delta_data_m*(z_p-z_val))/(z_p-z_m)
    return k_data_p , Delta_data
  

choose the value of z for the model

In [ ]:
mini = 1
z=7.9
for item in data:
   k,power_spectrum = extractPS(item[1]['data_PS'] , item[1]['z_PS']   ,z)
   if max(power_spectrum) > mini:
      mini = max(power_spectrum)
print(mini)

8810.183694081057


In [7]:
def interpxHglobal(xH_global , z_nodes):
  tck = interpolate.splrep(z_nodes , xH_global)
  result = interpolate.splev(5.9 , tck)
  result = result.item()
  return max(0 , result)

In [13]:
# default value
z = 10.4
dataDict = {}

for i,item in enumerate(data):
  mykey = tuple([item[1]['astro_params']['F_STAR10'] ,item[1]['astro_params']['ALPHA_STAR'], 
                 item[1]['astro_params']['F_ESC10'],item[1]['astro_params']['ALPHA_ESC'] 
                 ,item[1]['astro_params']['M_TURN'],item[1]['astro_params']['t_STAR']  
                 ,item[1]['astro_params']['L_X'] ,item[1]['astro_params']['NU_X_THRESH'], item[1]['astro_params']['X_RAY_SPEC_INDEX']])
  k,power_spectrum = extractPS(item[1]['data_PS'] , item[1]['z_PS']   ,z)
  xH = interpxHglobal(np.flip(np.array(item[1]['global_xH'])) , np.flip(np.array(item[1]['z_global'])))
  
  dataDict[i] = {'model params' :{'F_STAR10':mykey[0] ,'ALPHA_STAR': mykey[1], 'F_ESC10':mykey[2], 'ALPHA_ESC':mykey[3] 
                                  , 'M_TURN' :mykey[4],'t_STAR':mykey[5], 'L_X': mykey[6],  
                                  'NU_X_THRESH': mykey[7],'X_RAY_SPEC_INDEX': mykey[8]  } 
                 , 'ps': power_spectrum  , 'k' : k , 'z':z  , 'tau':item[1]['tau'].value , 'xH':xH}
print(dataDict[1])
  

{'model params': {'F_STAR10': -2.016854077611207, 'ALPHA_STAR': -0.2725655007830146, 'F_ESC10': -0.9861448680481955, 'ALPHA_ESC': -0.7479164763070929, 'M_TURN': 11.89360880050867, 't_STAR': 0.4841324953694949, 'L_X': 32.10228460361731, 'NU_X_THRESH': 681.5778183451747, 'X_RAY_SPEC_INDEX': 0.024163237231035417}, 'ps': array([7.62410981e-06, 1.87882016e-05, 5.42190130e-05, 1.37285518e-04,
       3.18801367e-04, 7.39819893e-04, 1.91083596e-03, 4.84431521e-03,
       1.01315176e-02, 1.77932284e-02, 2.72300391e-02]), 'k': array([0.03005976, 0.04508964, 0.06763446, 0.1014517 , 0.15217754,
       0.22826631, 0.34239947, 0.51359921, 0.77039881, 1.15559822,
       1.73339733]), 'z': 10.4, 'tau': 0.030026326805406692, 'xH': 0.9998198368231681}


In [9]:
print(len(dataDict))

27210


In [14]:
filename = '/content/drive/MyDrive/21cm project/data for training/trainingData_28000samples_all_params_131022_z=' + str(z)+'_.pk'
pickle.dump(dataDict,open(filename ,'wb'))

In [ ]:
z = 7.9
counter = 0
for item in data:
  mykey = tuple([item[1]['astro_params']['F_STAR10'] , item[1]['astro_params']['F_ESC10'] ,item[1]['astro_params']['L_X'],item[1]['astro_params']['M_TURN'],item[1]['astro_params']['NU_X_THRESH']], )
  k,power_spectrum = extractPS(item[1]['data_PS'] , item[1]['z_PS']   ,z)
  if np.array_equal([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],power_spectrum):
  
    counter+=1  
print(counter)
    

840


In [ ]:
fig = plt.figure(figsize = (10, 10))
ind = np.random.rand(100)*2000
ind = [int(i) for i in ind]

itms = list(dataDict.items())

for i in ind:
  plt.semilogx(itms[i][1]['k'], itms[i][1]['ps'], ls = '--', label = 'E_0 = ' + str(itms[i][1]['model params']['NU_X_THRESH'])+ ', L_X = ' + str(itms[i
                                                                                                                                                      ][1]['model params']['L_X']))
  #plt.scatter(item[1]['k'], item[1]['ps'], ls = '--', label = 'E_0 = ' + str(item[1]['model params']['NU_X_THRESH']))
  plt.xscale('log')
plt.xlabel('$k\,[1/\mathrm{Mpc}]$',fontsize=20)
plt.ylabel('$\Delta_{21}^2\,[\mathrm{mK}^2]\,\,(z=$'+str(7.9)+')',fontsize=15)
plt.legend()
plt.grid()
plt.show()
  





# for i, item in enumerate(dataDict.items()):
#     plt.semilogx(item[1]['k'], item[1]['ps'], ls = '--', label = 'E_0 = ' + str(item[1]['model params']['NU_X_THRESH'])+ ', L_X = ' + str(item[1]['model params']['L_X']))
#     plt.scatter(item[1]['k'], item[1]['ps'], ls = '--', label = 'E_0 = ' + str(item[1]['model params']['NU_X_THRESH']))
#     plt.xscale('log')
# plt.xlabel('$k\,[1/\mathrm{Mpc}]$',fontsize=20)
# plt.ylabel('$\Delta_{21}^2\,[\mathrm{mK}^2]\,\,(z=$'+str(7.9)+')',fontsize=15)
# plt.legend()
# plt.grid()
# plt.show()